In [ ]:
HyperResNet(input_shape=(128, 128, 3), classes = 10)

In [ ]:
class mlp_tuning (keras_tuner.HyperModel):

    def build_model (self, hyperParameter):
        model = tf.keras.models.Sequential()
        model.add(
            tf.keras.layers.Dense(
                units=hyperParameter.Int('units', min_value=178, max_value=218, step=10),
                input_dim=X_train.shape[1],
                kernel_initializer='glorot_uniform',
                activation='LeakyReLU'
            ))
        model.add(
            tf.keras.layers.Dense(
                units=hyperParameter.Int('units', min_value=29, max_value=69, step=10),
                kernel_initializer='glorot_uniform',
                activation='LeakyReLU'
            ))
        model.add(
            tf.keras.layers.Dense(
                units=1,
                kernel_initializer='glorot_uniform',
        ))
        # model.summary()
        model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
                    loss=rmspe)

    #     rlr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_delta=1e-5, verbose=1)
    #     es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=11, restore_best_weights=True, verbose=1)
    #     callback_list = [rlr, es]
    #     history = model.fit(X_train, y_train,
    #                     batch_size=500, epochs=1000, verbose=1,
    #                     validation_data=(X_val, y_val), callbacks=callback_list
    # )

        return model

    def fit (self, hyperparameter, model, x, y, validation_data, callbacks=None, **kwangs):

        batch_size = hyperparameter.Int("batch_size", 512, 1024, step=128, default=512)

        train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(
            batch_size
        )
        validation_data = tf.data.Dataset.from_tensor_slices(validation_data).batch(
            batch_size
        )
        optimizer = keras.optimizers.Adam(
            hyperparameter.Float("learning_rate", 1e-4, 1e-2, sampling="log", default=1e-3)
        )
        epoch_loss_metric = keras.metrics.Mean()

        # Function to run the train step.
        @tf.function
        def run_train_step(images, labels):
            with tf.GradientTape() as tape:
                logits = model(images)
                loss = loss_fn(labels, logits)
                # Add any regularization losses.
                if model.losses:
                    loss += tf.math.add_n(model.losses)
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

        # The metric to track validation loss.
        epoch_loss_metric = keras.metrics.Mean()

        # Function to run the validation step.
        @tf.function
        def run_val_step(images, labels):
            logits = model(images)
            loss = loss_fn(labels, logits)
            # Update the metric.
            epoch_loss_metric.update_state(loss)

        # Assign the model to the callbacks.
        for callback in callbacks:
            callback.model = model

        # Record the best validation loss value
        best_epoch_loss = float("inf")

        # The custom training loop.
        for epoch in range(2):
            print(f"Epoch: {epoch}")

            # Iterate the training data to run the training step.
            for images, labels in train_ds:
                run_train_step(images, labels)

            # Iterate the validation data to run the validation step.
            for images, labels in validation_data:
                run_val_step(images, labels)

            # Calling the callbacks after epoch.
            epoch_loss = float(epoch_loss_metric.result().numpy())
            for callback in callbacks:
                # The "my_metric" is the objective passed to the tuner.
                callback.on_epoch_end(epoch, logs={"my_metric": epoch_loss})
            epoch_loss_metric.reset_states()

            print(f"Epoch loss: {epoch_loss}")
            best_epoch_loss = min(best_epoch_loss, epoch_loss)

        # Return the evaluation metric value.
        return best_epoch_loss

In [ ]:
def build_model (hyperParameter):
    model = tf.keras.models.Sequential()
    model.add(
        tf.keras.layers.Dense(
            units=hyperParameter.Int('units', min_value=178, max_value=218, step=10),
            input_dim=X_train.shape[1],
            kernel_initializer='glorot_uniform',
            activation='LeakyReLU'
        ))
    model.add(
        tf.keras.layers.Dense(
            units=hyperParameter.Int('units', min_value=29, max_value=69, step=10),
            kernel_initializer='glorot_uniform',
            activation='LeakyReLU'
        ))
    model.add(
        tf.keras.layers.Dense(
            units=1,
            kernel_initializer='glorot_uniform',
    ))
    # model.summary()
    model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
                  loss=rmspe)

#     rlr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_delta=1e-5, verbose=1)
#     es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=11, restore_best_weights=True, verbose=1)
#     callback_list = [rlr, es]
#     history = model.fit(X_train, y_train,
#                     batch_size=500, epochs=1000, verbose=1,
#                     validation_data=(X_val, y_val), callbacks=callback_list
# )

    return model


In [ ]:
tuner = kerastuner.Hyperband(hypermodel = build_model,
                             objective = 'val_accuracy',
                             directory = 'my_dir',
                             max_epochs= 100,
                             ba
                             )

In [ ]:
rlr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_delta=1e-5, verbose=1)
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=11, restore_best_weights=True, verbose=1)
callback_list = [rlr, es]

tuner.search(X_train, y_train, epochs=300, validation_data=(X_val, y_val), callbacks=callback_list)